# 전체 폴더 처리 코드(순회)

In [ ]:
import os
import json
import pandas as pd

def find_json_files(root_folder):
    json_files = [os.path.join(root, file) for root, _, files in os.walk(root_folder) for file in files if file.endswith(".json")]
    print(f"[DEBUG] Found {len(json_files)} JSON files in {root_folder}")
    return json_files

def extract_skeleton_points(json_data):
    for annotation in json_data.get("annotations", []):
        if annotation.get("class") == "person" and "points" in annotation:
            points = annotation["points"]
            if isinstance(points, list) and len(points) % 3 == 0:
                return [(points[i], points[i+1]) for i in range(0, len(points), 3)]
            if all(isinstance(pt, (list, tuple)) and len(pt) == 2 for pt in points):
                return points
    return None

def is_impact_front_view(points):
    try:
        left_shoulder_x, right_shoulder_x = points[5][0], points[2][0]
        left_wrist_x, right_wrist_x = points[9][0], points[6][0]
        return right_shoulder_x <= right_wrist_x <= left_shoulder_x and right_shoulder_x <= left_wrist_x <= left_shoulder_x
    except (TypeError, IndexError):
        print(f"[DEBUG] Invalid points format: {points}")
        return False

def is_impact_side_view(points, address_right_wrist, address_left_wrist):
    try:
        right_wrist, left_wrist = points[6], points[9]
        if not (address_right_wrist and address_left_wrist):
            return False
        return (abs(right_wrist[0] - address_right_wrist[0]) + abs(right_wrist[1] - address_right_wrist[1]) +
                abs(left_wrist[0] - address_left_wrist[0]) + abs(left_wrist[1] - address_left_wrist[1])) < 150  # Threshold increased
    except (TypeError, IndexError):
        print(f"[DEBUG] Invalid points format for side view: {points}")
        return False

def extract_frame_number(filename):
    try:
        frame_str = os.path.splitext(os.path.basename(filename))[0].split('_')[-1]
        return int(frame_str)
    except ValueError:
        print(f"[DEBUG] Cannot extract frame number from {filename}")
        return None

def get_title_prefix(filename):
    parts = os.path.splitext(os.path.basename(filename))[0].split('_')
    return '_'.join(parts[:-1]) if len(parts) > 1 else parts[0]

def process_json_files(root_folder, output_csv, cluster_gap_threshold=5, margin=5):  # Margin increased
    json_files = find_json_files(root_folder)
    groups = {}
    
    for file in json_files:
        title, frame_number = get_title_prefix(file), extract_frame_number(file)
        if frame_number is not None:
            groups.setdefault(title, []).append((frame_number, file))
    
    print(f"[DEBUG] Grouped files into {len(groups)} unique titles")
    
    impact_filenames = []
    for title, files_info in groups.items():
        files_info.sort()
        
        clusters, current_cluster = [], [files_info[0]]
        for info in files_info[1:]:
            if info[0] - current_cluster[-1][0] <= cluster_gap_threshold:
                current_cluster.append(info)
            else:
                clusters.append(current_cluster)
                current_cluster = [info]
        clusters.append(current_cluster)
        
        if not clusters:
            continue
        
        with open(clusters[0][0][1], "r", encoding="utf-8") as f:
            address_points = extract_skeleton_points(json.load(f))
        if not address_points or len(address_points) < 10:
            print(f"[DEBUG] Address points missing or invalid in {clusters[0][0][1]}")
            continue
        
        address_right_wrist = address_points[6]
        address_left_wrist = address_points[9]
        
        detected_impacts = []
        for cluster in clusters[1:]:
            impact_count = 0
            total = len(cluster)
            for _, file in cluster:
                with open(file, "r", encoding="utf-8") as f:
                    points = extract_skeleton_points(json.load(f))
                if not points:
                    continue
                if is_impact_front_view(points) or is_impact_side_view(points, address_right_wrist, address_left_wrist):
                    impact_count += 1
                    detected_impacts.append(file)
            print(f"[DEBUG] Cluster {title}: {impact_count}/{total} impact frames detected")
        
        if detected_impacts:
            impact_filenames.extend(detected_impacts[-int(len(detected_impacts) / 2):])  # Keep only second half
    
    pd.DataFrame(impact_filenames, columns=["filename"]).to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"[DEBUG] Impact 조건을 만족하는 {len(impact_filenames)}개의 JSON 파일 목록이 {output_csv}에 저장되었습니다.")

# 실행 예시
root_folder = r"D:\\golfDataset\\스포츠 사람 동작 영상(골프)\\Training"
output_csv = "impact_filenames.csv"
process_json_files(root_folder, output_csv)

# 특정 폴더 처리 코드(시간절약, 테스트)

In [1]:
import os
import json
import pandas as pd

def find_json_files(root_folder):
    """ 주어진 폴더 내의 JSON 파일만 반환한다. """
    return [
        os.path.join(root_folder, file)
        for file in os.listdir(root_folder)
        if os.path.isfile(os.path.join(root_folder, file)) and file.lower().endswith(".json")
    ]

def extract_skeleton_points(json_data):
    """ JSON에서 class가 person인 객체의 points 값을 가져오고, (x, y, visibility) 형식이면 (x, y)만 추출한다. """
    for annotation in json_data.get("annotations", []):
        if annotation.get("class") == "person" and "points" in annotation:
            points = annotation["points"]
            
            # Mediapipe 스타일의 (x, y, visibility) 리스트일 경우 (x, y)만 추출
            if isinstance(points, list) and all(isinstance(pt, (int, float)) for pt in points):
                if len(points) % 3 == 0:  # (x, y, visibility) 구조인지 확인
                    return [(points[i], points[i+1]) for i in range(0, len(points), 3)]
            
            # (x, y) 형태인지 확인
            if all(isinstance(pt, (list, tuple)) and len(pt) == 2 for pt in points):
                return points
            
            print(f"Invalid points format: {points}")
            return None
    return None

def is_impact_front_view(points):
    """ 정면 및 유사 방향에서의 impact 판단: 양쪽 손목이 양쪽 어깨의 x좌표 내에 존재하는지 확인 """
    if not points or len(points) < 16:
        return False
    
    try:
        left_shoulder_x, right_shoulder_x = points[4][0], points[3][0]
        left_wrist_x, right_wrist_x = points[8][0], points[7][0]
    except (TypeError, IndexError) as e:
        print(f"Error accessing points: {points} - {e}")
        return False
    
    return right_shoulder_x <= right_wrist_x <= left_shoulder_x and right_shoulder_x <= left_wrist_x <= left_shoulder_x

def is_impact_side_view(points, address_right_wrist, address_left_wrist):
    """ 측면 방향에서의 impact 판단: 현재 프레임의 손목 위치와 어드레스 시점의 손목 위치 차이가 작으면 impact로 간주 """
    if not points or len(points) < 16:
        return False
    
    right_wrist, left_wrist = points[7], points[8]
    if not (address_right_wrist and address_left_wrist):
        return False
    
    right_diff = abs(right_wrist[0] - address_right_wrist[0]) + abs(right_wrist[1] - address_right_wrist[1])
    left_diff = abs(left_wrist[0] - address_left_wrist[0]) + abs(left_wrist[1] - address_left_wrist[1])
    
    return (right_diff + left_diff) < 50  # 임계값 미만이면 impact

def extract_frame_number(filename):
    """
    파일 이름에서 마지막 4자리 숫자를 추출한다.
    예: 20201124_General_035_DOS_A_M40_BT_001_0001.json -> 1 반환
    """
    basename = os.path.basename(filename)
    name_without_ext = os.path.splitext(basename)[0]
    parts = name_without_ext.split('_')
    if parts:
        frame_str = parts[-1]
        try:
            return int(frame_str)
        except ValueError:
            print(f"Cannot convert {frame_str} to int in filename {filename}")
            return None
    return None

def get_title_prefix(filename):
    """
    파일 이름에서 마지막 4자리 숫자(프레임 번호)를 제외한 제목(prefix)을 반환한다.
    예: 20201124_General_035_DOS_A_M40_BT_001_0001.json -> 20201124_General_035_DOS_A_M40_BT_001
    """
    basename = os.path.basename(filename)
    name_without_ext = os.path.splitext(basename)[0]
    parts = name_without_ext.split('_')
    if len(parts) > 1:
        return '_'.join(parts[:-1])
    return name_without_ext

def process_json_files(root_folder, output_csv, cluster_gap_threshold=5, margin=2):
    """
    - json 파일을 파일명(prefix)별로 그룹화하여,
      한 동작에서 어드레스와 임팩트를 구분한다.
    - 각 그룹 내에서 프레임 번호가 연속(갭이 cluster_gap_threshold 이하인 경우)하는 것을 하나의 클러스터로 판단.
    - 첫번째 클러스터는 어드레스(주소)로 간주하여 CSV에 저장하지 않고,
      이후 클러스터 중 과반수 프레임이 impact 조건(정면 혹은 측면)을 만족하면 impact 클러스터로 판단.
    - impact 클러스터에 대해서는 양쪽 끝에서 margin 만큼 추가 프레임도 포함시킨다.
    """
    json_files = find_json_files(root_folder)
    # 그룹화: title_prefix -> list of (frame_number, full_path, basename)
    groups = {}
    for file in json_files:
        title = get_title_prefix(file)
        frame_number = extract_frame_number(file)
        if frame_number is None:
            continue
        groups.setdefault(title, []).append((frame_number, file, os.path.basename(file)))
    
    impact_filenames = []
    
    # 각 그룹별 처리
    for title, files_info in groups.items():
        # 프레임 번호 기준 정렬
        files_info.sort(key=lambda x: x[0])
        
        # 클러스터 생성 (연속성 기준: 인접 프레임 번호 차이가 cluster_gap_threshold 이하)
        clusters = []
        current_cluster = [files_info[0]]
        for info in files_info[1:]:
            if info[0] - current_cluster[-1][0] <= cluster_gap_threshold:
                current_cluster.append(info)
            else:
                clusters.append(current_cluster)
                current_cluster = [info]
        clusters.append(current_cluster)
        
        if not clusters:
            continue
        
        # 첫번째 클러스터는 어드레스(주소)로 간주
        address_cluster = clusters[0]
        # 어드레스 시점의 손목 좌표는 해당 클러스터의 첫번째 파일에서 추출
        with open(address_cluster[0][1], "r", encoding="utf-8") as f:
            data = json.load(f)
        address_points = extract_skeleton_points(data)
        if address_points and len(address_points) >= 9:
            address_right_wrist, address_left_wrist = address_points[7], address_points[8]
        else:
            address_right_wrist, address_left_wrist = None, None
        
        # 두번째 클러스터부터 impact 여부 판단
        for cluster in clusters[1:]:
            impact_count = 0
            total = len(cluster)
            for _, file_path, _ in cluster:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                points = extract_skeleton_points(data)
                if not points:
                    continue
                if is_impact_front_view(points) or is_impact_side_view(points, address_right_wrist, address_left_wrist):
                    impact_count += 1
            # 과반수 이상이 impact 조건을 만족하면 해당 클러스터를 impact 클러스터로 판단
            if impact_count >= total / 2:
                # 클러스터의 최소, 최대 프레임 번호로 확장(여유 margin 추가)
                start_frame = cluster[0][0]
                end_frame = cluster[-1][0]
                extended_cluster = []
                # 같은 그룹 내에서 margin 범위 내의 모든 프레임을 포함 (이미 포함된 클러스터와 중복되지 않도록)
                for info in files_info:
                    if start_frame - margin <= info[0] <= end_frame + margin:
                        extended_cluster.append(info)
                for _, _, fname in extended_cluster:
                    if fname not in impact_filenames:
                        impact_filenames.append(fname)
            # 만약 후반에 나타난 클러스터가 address로 예측된다면, 따로 impact로 추가하지 않음.
    
    df = pd.DataFrame(impact_filenames, columns=["filename"])
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"Impact 조건을 만족하는 JSON 파일 목록이 {output_csv}에 저장되었습니다.")

# 실행 예시
root_folder = r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Amateur\male\[라벨]swing_03\20201123_General_002_DIS_S_M20_SS"
output_csv = "impact_filenames.csv"
process_json_files(root_folder, output_csv)


Impact 조건을 만족하는 JSON 파일 목록이 impact_filenames.csv에 저장되었습니다.


In [9]:
import os
import shutil
import pandas as pd

def copy_matching_images(csv_file, source_folder, target_folder):
    """ impact_filenames.csv에 기록된 파일과 동일한 이름의 jpg 파일을 images 폴더로 복사 (하위 폴더 포함) """
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # CSV 파일에서 파일 이름 읽기
    df = pd.read_csv(csv_file, encoding="utf-8-sig")
    
    if "filename" not in df.columns:
        raise KeyError("CSV 파일에 'filename' 열이 없습니다.")

    df["filename"] = df["filename"].astype(str).str.strip()
    
    # 파일명을 추출하여 JSON → JPG 변환
    json_filenames = set(os.path.basename(x).replace(".json", ".jpg") for x in df["filename"])

    print(f"[DEBUG] CSV에서 {len(json_filenames)}개의 파일명을 로드했습니다.")
    print(f"[DEBUG] JSON 변환 후 파일 목록 예시: {list(json_filenames)[:5]}")

    copied_files = 0
    missing_files = []

    # 재귀적으로 모든 하위 폴더 탐색
    for root, _, files in os.walk(source_folder):
        for file in files:
            if os.path.basename(file) in json_filenames:  # 파일명만 비교
                source_path = os.path.join(root, file)
                target_path = os.path.join(target_folder, file)
                
                shutil.copy(source_path, target_path)
                copied_files += 1
            else:
                missing_files.append(file)

    print(f"[INFO] 총 {copied_files}/{len(json_filenames)}개의 파일이 복사되었습니다.")

# 실행 예시
source_folder = r"D:\\golfDataset\\스포츠 사람 동작 영상(골프)\\Training"
target_folder = r"D:\\golfDataset\\preprocessing\\impact_images"
csv_file = "impact_filenames.csv"

copy_matching_images(csv_file, source_folder, target_folder)


[DEBUG] CSV에서 13개의 파일명을 로드했습니다.
[DEBUG] JSON 변환 후 파일 목록 예시: ['20201124_General_035_DOS_A_M40_BT_062_0113.jpg', '20201124_General_035_DOS_A_M40_BT_062_0114.jpg', '20201124_General_035_DOS_A_M40_BT_062_0110.jpg', '20201124_General_035_DOS_A_M40_BT_062_0107.jpg', '20201124_General_035_DOS_A_M40_BT_062_0111.jpg']
[INFO] 총 13/13개의 파일이 복사되었습니다.
